In [1]:
%matplotlib inline

import os, re, json

import numpy as np
import pandas as pd

Loading tha raw data scraped from Ouedkniss

In [2]:
df_full = pd.read_json(os.path.abspath('dataset/cars_raw.json'))

In [3]:
df_full.tail()

,id,date,category,energy,engine,range,color,papers,options,transmission,price,title,address
17581,26996430,06-06-2021 à 20:06,Break - Familiale,Diesel,1.6 ess 102ch,160000 km,Blanc,Carte Grise (safia),"Climatisation , ABS , ESP , Radar de recul...",Manuelle,275 Millions Offert,Volkswagen Caddy Confortline 2017,Alger Sidi moussa
17582,27136575,06-06-2021 à 16:53,Mini citadine,Essence,1.1 ess 68ch,223000 km,Gris,Carte Grise (safia),Climatisation,Manuelle,63 Millions Fixe,Chery QQ 2012,Bordj bou arreridj Bordj bou arreridj
17583,27136210,06-06-2021 à 16:24,Berline,Diesel,2.0 TDCI 121ch,200000 km,Gris Argent,Carte Grise (safia),"Climatisation , ABS , ESP , Retroviseurs é...",Manuelle,128 Millions Offert,Chevrolet Optra 4 portes 2012,Alger Bordj el kiffan
17584,27136039,06-06-2021 à 16:47,Citadine,Diesel,1.6 HDI 92ch,200 km,Blanc,Carte Grise (safia),"Climatisation , ABS , ESP , Radar de recul...",Manuelle,111 Millions Négociable,Peugeot 207 Allure 2011,Oum el bouaghi Ain mlila
17585,27135969,06-06-2021 à 16:55,Berline,Essence,1.6 ess 16V 109ch,331000 km,Gris,Carte Grise (safia),"Climatisation , Direction assisstée , Retro...",Manuelle,65 Millions Offert,Chevrolet Optra 4 portes 2006,Alger Belouizdad


In [4]:
df = df_full.copy()

Loading Algeria provinces list, in addition to cars brands list scraped from Ouedkniss

In [5]:
with open(os.path.abspath("wilayas.json"), 'r') as wilayas_file:
    wilayas = json.load(wilayas_file)
with open(os.path.abspath("brands.json"), 'r') as brands_file:
    brands = json.load(brands_file)

Some addresses contain empty strings, we will transform them to NaN value

In [6]:
df['address'] = df['address'].map(lambda x: np.nan if x == '' else x)

We will then preprocess obvious values like price, the type of offer and the car range ...

For the date attribute for instance, we will only keep the date, the time part is irrelevant.

In [7]:
df['offer'] = df['price'].map(lambda x: x.split()[-1].lower(), na_action = 'ignore')
df['price'] = df['price'].map(lambda x: float(x.split()[0]), na_action = 'ignore')
df['range'] = df['range'].map(lambda x: int(x.split()[0]), na_action = 'ignore')
df['color'] = df['color'].map(lambda x: str(x).split()[0].lower(), na_action = 'ignore')
df['date'] = df['date'].map(lambda x: str(x).split()[0], na_action = 'ignore')
df['category'] = df['category'].map(lambda x: x.lower())
df['energy'] = df['energy'].map(lambda x: x.lower())
df['papers'] = df['papers'].map(lambda x: x.lower())
df['transmission'] = df['transmission'].map(lambda x: x.lower())

Using the list of provinces, we will proceed to split the address column to wilaya and town

In [8]:
def split_address(address):
    # Checking for NaN values
    if pd.isna(address):
        return np.nan, np.nan
    
    address = address.lower().split()
    for i in range(1, len(address) + 1):
        if ' '.join(address[ : i]) in wilayas:
            town = ' '.join(address[i : ]).split(',')[0].strip()
            return (' '.join(address[ : i]), town)
    return np.nan, np.nan

In [9]:
df[['wilaya', 'town']] = pd.DataFrame(df['address'].map(split_address).tolist())

In [10]:
df.drop('address', axis = 1, inplace = True)

Brand is the most important feature for price prediciton, yet it is the most difficult to prepare.

Users have introduced many unknown brands, due to spelling errors or using other languages, some brands are not even car brands.

In [11]:
def split_title(title):
    # Checking for NaN values
    if pd.isna(title):
        return np.nan, np.nan, np.nan
    
    title = title.lower().split()
    for i in range(1, len(title) + 1):
        if ' '.join(title[ : i]) in brands:
            return (' '.join(title[ : i]), ' '.join(title[i : i + 1]), int(title[-1]))
    return (title[0], ' '.join(title[1 : -1]), int(title[-1]))

In [12]:
df[['brand', 'model', 'year']] = pd.DataFrame(df['title'].map(split_title).tolist())

brands_counts = df['brand'].value_counts().sort_values(
            ascending = False).reset_index().set_axis(
            ['brand', 'count'], axis = 1)

We will try to find any patterns in unknown brands.

In [13]:
unknown_brands = []
for i in range(len(brands_counts)):
    if brands_counts.iloc[i]['brand'] not in brands:
        unknown_brands.append(brands_counts.iloc[i]['brand'])
print(' '.join(unknown_brands))

clio huanghai range golf 207 wuling iveco ibiza r4 jinbei سامبول rono maruti sym بيجو alto willing changhe jin moto mpm atos symbol xzara 309 إسيزي sandero شيفروليه spark qq peugot رونو maruté ستبواى citroën xara هاربين wulling espace 206 foord ivico sandiro sokon changh dfac بولمهار j tisane bartner vosvaken begout jita بيكانتو logan ايبيزا simca mercedes كليوا harbin ezgo sonalika changane 50 laguna new سيتروان hafei compiss niva city إبيزا bolo tous شيفرولي جيط polo motos-scooters ليون fyat q5 405 hafi 504 golf6 شراد jonway changhe. mazouz lincoln okinoi vigo herbin piaggio sonacom reunelt renualt corola سبارك stepwaw cf arrow shineray bijo سوزوكي location stepway haima7


We can observe 3 special cases:

1. Brands written in Arabic: the only way to recover those is translating every Arabic title, which will end up taking alot of time.

2. Models names misteken with brands, e.g. ibiza, range rover, atos ... , brands names with spelling errors or actual brands that are not already in the list.

3. Brands that are not car brands at all (motocyles ...etc). Thoes are irrelevant.

In [14]:
len(df[df['brand'].isin(unknown_brands)])

149

There are 149 cars with unknown brands. It is not that significant number so we will drop all.

In [15]:
for index, car in df.iterrows():
    if car['brand'] in unknown_brands:
        df.drop(index, axis = 0, inplace = True)

In [16]:
len(df)

17437

Now we have to deal with the model similarly.

It is much more difficult to grasp every model for every brand, so we will be removing models with number of occurrences equals or under 3 occurrences.

In [17]:
models = df['model'].value_counts().reset_index().set_axis(['model', 'count'], axis = 1)
len(models[models['count'] == 1]) + 2 * len(models[models['count'] == 2]) + 3 * len(models[models['count'] == 3])

785

Luckily, there is relatively few of them.

And since they are rare individuals, models would have low confidence evaluating similar occurrences.

In [18]:
dropped_models = models[models['count'].isin([1, 2])]['model'].tolist()

In [19]:
for index, car in df.iterrows():
    if car['model'] in dropped_models:
        df.drop(index, axis = 0, inplace = True)

In [20]:
len(df)

16832

In [21]:
df.drop('title', axis = 1, inplace = True)

Now for the engine, we will try to extract characteristics of the engine from the string as follow:

In [22]:
def extract_engine_charact(engine):
    if pd.isna(engine):
        return np.nan, np.nan
    
    try:
        vol = re.search('^(\d+\.\d+)', engine).group(1)
    except Exception:
        vol = np.nan
    try:
        ch = re.search('(\d+)ch', engine).group(1)
    except Exception:
        ch = np.nan
    return float(vol), float(ch)

In [23]:
df[['volume', 'horses']] = pd.DataFrame(df['engine'].map(extract_engine_charact).tolist())

In [24]:
df['volume'].value_counts().sort_index()

0.400        4
0.600        6
0.700        1
0.800      419
0.900       18
1.000      494
1.100      783
1.200     1356
1.250       23
1.300      525
1.400     1512
1.500     1933
1.600     2533
1.700      101
1.800      303
1.900     1734
2.000     2310
2.100        9
2.200      351
2.219        1
2.300        5
2.400       11
2.500      414
2.560        1
2.700        9
2.800       31
2.900        3
3.000       42
3.200        1
3.500        3
3.600        2
4.200       28
4.400        2
5.000        1
20.150       1
Name: volume, dtype: int64

There is one outlier with 20.150, which seems more likely to be 2.0 and 150 horses power.

In [25]:
df.loc[df['volume'] == 20.150, ['volume', 'horses']] = [2.0, 150.0]

In [26]:
df['volume'] = np.round(df['volume'], 1)

In [27]:
df.drop('engine', axis = 1, inplace = True)

We will then try to one hot encode the option feature.

In [28]:
df.iloc[0]['options']

' Climatisation ,  Toit ouvrant ,  ABS ,  ESP ,  Radar de recul ,  Direction assisstée ,  Retroviseurs électriques ,  Phares antibrouillard ,  Alarme ,  Phares xénon ,  Jantes Alliage ,  Feux du jour ,  Vitres éléctriques '

In [29]:
options = set()
for index, car in df.iterrows():
    if not pd.isna(car['options']):
        car_options = re.split(' +, +', car['options'].lower().strip())
        options.update(car_options)
options

{'abs',
 'alarme',
 'climatisation',
 'direction assisstée',
 'esp',
 'feux du jour',
 'jantes alliage',
 'phares antibrouillard',
 'phares xénon',
 'radar de recul',
 'retroviseurs électriques',
 'toit ouvrant',
 'vitres éléctriques'}

In [30]:
for option in options:
    df[option] = 0

In [31]:
for index, car in df.iterrows():
    if not pd.isna(car['options']):
        car_options = re.split(' +, +', car['options'].lower().strip())
        df.loc[index, car_options] = 1

In [32]:
df.drop('options', axis = 1, inplace = True)

Our dataset is finally ready for some exploration.

In [33]:
df.head()

,id,date,category,energy,range,color,papers,transmission,price,offer,...,direction assisstée,climatisation,jantes alliage,esp,vitres éléctriques,retroviseurs électriques,phares antibrouillard,phares xénon,feux du jour,abs
0,27661724,27-07-2021,moyenne berline,diesel,56000,blanc,carte grise (safia),manuelle,410.0,négociable,...,1,1,1,1,1,1,1,1,1,1
1,27661710,27-07-2021,commerciale,essence,2,gris,carte grise (safia),manuelle,138.0,fixe,...,0,0,0,0,0,0,0,0,0,0
2,27661707,27-07-2021,citadine,diesel,10,gris,carte grise (safia),manuelle,305.0,négociable,...,1,1,0,0,1,1,1,0,0,1
3,27661699,27-07-2021,citadine,essence,277000,blanc,carte grise (safia),manuelle,150.0,offert,...,1,1,1,1,1,1,0,0,0,1
4,27661693,27-07-2021,mini citadine,essence,18000,bleu,carte grise (safia),manuelle,63.0,fixe,...,0,0,0,0,0,0,0,0,0,0


In [34]:
df.to_csv(os.path.abspath('dataset/cars_cleaned.csv'), index = False)